In [2]:
import torch
#Use this space for manually computing the scaled dot product attention

In [4]:
#This is how you'll do diagrams for tensors from here on out
import pandas as pd
#pd.DataFrame(input)

def printTensor(tensor):
    df = pd.DataFrame(tensor)
    print(df)

In [21]:
input = torch.tensor([[-1.5898,1.5792,0.8666,1.0245],[2.2796,0.9813,1.4023,-0.6616],[0.1588,0.4028,-1.7044,-1.8284],[0.4366,-1.0197,-0.4217, 1.4877],[-0.8311,-0.1762,-1.8709,2.2313]], dtype=torch.float32)
input
#Paper example input from embedding layers

tensor([[-1.5898,  1.5792,  0.8666,  1.0245],
        [ 2.2796,  0.9813,  1.4023, -0.6616],
        [ 0.1588,  0.4028, -1.7044, -1.8284],
        [ 0.4366, -1.0197, -0.4217,  1.4877],
        [-0.8311, -0.1762, -1.8709,  2.2313]])

In [22]:
#Lets try to initialize the K,Q,V wieghts
import torch.nn as nn
import torch.nn.init as init
d_model = 16
input = nn.Linear(4, d_model)(input) # We need to reshape the input to be compatible for multiplication with our QKV weights
input = input.reshape(1,5,d_model)
# Our example input has a column dimension of 4, or 4 featues in the sequence. 
# This is important for the linear layers


# Set dimensions of the weights
W_Q = nn.Linear(d_model, d_model, bias=False)
W_K = nn.Linear(d_model, d_model, bias=False)
W_V = nn.Linear(d_model, d_model, bias=False)

nn.init.kaiming_normal_(W_Q.weight)
nn.init.kaiming_normal_(W_K.weight)
nn.init.kaiming_normal_(W_V.weight)

with torch.no_grad():
    print("WQ matrix:")
    printTensor(W_Q.weight)
    print("\nWK matrix:")
    printTensor(W_K.weight)
    print("\nWV matrix:")
    printTensor(W_V.weight)



WQ matrix:
          0         1         2         3         4         5         6   \
0  -0.470987  0.079266  0.174696 -0.535052 -0.012049  0.214860  0.068636   
1   0.648801 -0.012278 -0.040407  0.191615  0.118053  0.371963 -0.025763   
2   0.173569  0.121498 -0.344515  0.109018 -0.084727  0.547651 -0.370525   
3  -0.326787  0.244349 -0.194039  0.019395  0.780593  0.242292  0.798896   
4  -0.283382  0.002806 -0.918048 -0.164136 -0.034875  0.337081  0.147311   
5   0.628372  0.204924  0.044404 -0.050840  0.233980 -0.442214 -0.155582   
6   0.270670  0.028798 -0.188753  0.567402  0.346777  0.162149  0.493569   
7  -0.485668  0.380601  0.668065  0.420029  0.020114 -0.086267 -0.316658   
8   0.405937 -0.668566  0.195740  0.096004 -0.687596 -0.412855  0.152536   
9  -0.371967 -0.094910  0.205214 -0.152070 -0.425743  0.346095  0.047952   
10 -0.636151  0.142995 -0.367989 -0.007011 -0.034609 -0.059427 -0.011449   
11  0.347692  0.343580 -0.240791 -0.123103  0.200509  0.173337 -0.224128   
1

In [24]:
import math
# Lets compute Q,K,V first linear projection
with torch.no_grad():
    Q = W_Q(input)
    K = W_K(input)
    V = W_V(input)
    print("Q matrix:")
    printTensor(Q.numpy()[0, :, :]) # Changing print statements so I can see only the last two dimensions
    print("\nK matrix:")
    printTensor(K.numpy()[0, :, :])
    print("\nV matrix:",)
    printTensor(V.numpy()[0, :, :])
    

Q matrix:
         0         1         2         3         4         5         6   \
0  1.197957 -1.269360 -0.055508 -2.128647  0.974117 -0.342524 -2.896082   
1  0.167170 -2.022422  1.035629  2.415249  3.104412 -0.894096  0.676284   
2 -2.828585  1.758431 -0.259630 -1.649939  0.475646  0.315709  2.531099   
3  1.219062 -0.773609  1.505667 -0.123691  0.331173  0.037534 -1.679208   
4  1.114348  0.173057  0.744262 -2.361209 -0.294234  0.531684 -2.995847   

         7         8         9         10        11        12        13  \
0 -1.332305  0.830155  0.478851  1.966173  0.563795 -0.038380  1.905199   
1 -1.662630  1.382159 -1.976950 -0.751531 -0.135693  1.560863 -3.175181   
2 -0.658981  1.895863  0.125164  1.259131  0.729176  0.542601  2.224545   
3  0.163091 -1.108362  0.084641 -0.255893  0.493218 -0.360399 -1.593347   
4  0.302378 -1.510241  0.732952  1.940135  1.477124 -1.475554  1.387943   

         14        15  
0 -0.844068  0.294967  
1 -0.222186 -0.119694  
2 -0.819916 -1.8

ValueError: Must pass 2-d input. shape=(1, 5, 16)

In [21]:
# Scaled dot-product    
with torch.no_grad():
    attention_weights = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_model)
    print("Scaled Dot Product:")
    printTensor(attention_weights)

Scaled Dot Product:
          0         1         2         3         4
0  0.746611  0.760072  0.483585  0.555763  2.251897
1  3.714852 -6.536989  1.316062 -0.318690  5.196238
2 -0.382909  1.248583 -1.181069 -1.088685 -3.904110
3 -1.515646  0.781833  0.291480 -0.311119 -1.935483
4 -3.125703  4.380373 -0.287978 -1.128130 -6.200011


In [22]:
with torch.no_grad():
    #softmax
    attention_weights = torch.softmax(attention_weights, dim=-1)
    print("Softmax:")
    printTensor(attention_weights)

Softmax:
          0         1         2         3         4
0  0.123244  0.124914  0.094740  0.101832  0.555269
1  0.181567  0.000006  0.016491  0.003216  0.798720
2  0.141144  0.721459  0.063537  0.069687  0.004173
3  0.047541  0.472992  0.289665  0.158559  0.031242
4  0.000542  0.986179  0.009258  0.003996  0.000025


In [23]:
with torch.no_grad():
    #Last linear layer
    print("Output:")
    output = torch.matmul(attention_weights, V)
    printTensor(output)

Output:
          0         1         2         3
0 -0.330422  0.354216  0.240682 -0.287233
1 -0.704020  0.533120  0.187559 -1.103511
2  1.005840  0.509922  1.177463 -0.027600
3  0.636917  0.420271 -0.162054  1.025237
4  1.426986  0.317784  1.805311  0.754292


In [24]:
# Applying layer normalization to a mini batch
# Remember d_model is our embedding dimension
dropout = nn.Dropout(p=0.1)
layer_norm = nn.LayerNorm(output.size()[-1])
# We now will call the norm layer with our output
output = dropout(output)
output = layer_norm(output)
print(output)

tensor([[-0.9653,  1.3337,  0.5968, -0.9653],
        [-0.6559,  1.2212,  0.6968, -1.2621],
        [ 0.7226, -0.3331,  1.0880, -1.4775],
        [ 1.2931,  0.6150, -1.2077, -0.7005],
        [ 0.6077, -1.3133,  1.2630, -0.5573]],
       grad_fn=<NativeLayerNormBackward0>)


In [16]:
with torch.no_grad():
    printTensor(output)

          0         1         2         3
0  1.062668 -0.411011 -1.441476  0.789819
1  0.320404 -0.892024  1.497872 -0.926251
2  1.378798  0.523056 -1.039870 -0.861984
3  1.616217 -0.982255 -0.650158  0.016196
4  1.342489 -0.563447 -1.277423  0.498380


In [17]:
# a class for making multihead attetnion sublayer easier
import torch
import torch.nn as nn
import math

class MultiheadAttentionSublayer(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.linears = nn.ModuleList([nn.Linear(embed_dim, embed_dim) for _ in range(3)])
        self.layer_norm = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, x, attn_mask):
        q, k, v = [l(x) for l in self.linears]
        x, attn = self.attention(q, k, v, attn_mask)
        x = x + self.dropout(x)
        x = self.layer_norm(x)
        return x, attn

    def attention(self, q, k, v, attn_mask):
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(k.size(-1))
        if attn_mask is not None:
            scores = scores.masked_fill(attn_mask, -1e9)
        attn = torch.softmax(scores, dim=-1)
        x = torch.matmul(attn, v)
        return x, attn
        
model = MultiheadAttentionSublayer(d_model, 6)
attn_mask = None
attention_weights, weighted_output = model(input, attn_mask) # Compute the output of the forward method
print(weighted_output.shape) # Print the output tensor to verify it
print(input.shape)
print(d_model)


torch.Size([5, 5])
torch.Size([5, 4])
4


In [25]:
# Lets do the last step in the encoder - feed forward
import torch.nn as nn

# Define the feedforward layer
feedforward = nn.Sequential(
    nn.Linear(output.size()[-1], 16),
    nn.ReLU(),
    nn.Linear(16, d_model)
)

# Pass the normalized tensor through the feedforward layer
encoder_output = feedforward(output)

In [26]:
with torch.no_grad():
    printTensor(encoder_output)

          0         1         2         3
0 -0.158562  0.339174 -0.396286 -0.223677
1 -0.133566  0.335864 -0.438792 -0.219981
2  0.209002  0.020946 -0.521161 -0.132638
3  0.320134  0.019025 -0.788149 -0.231824
4  0.412192 -0.240543 -0.453010  0.024202


In [27]:
# Apply another norm layer to finishthe encoder!!!!
# Applying layer normalization to a mini batch
# Remember d_model is our embedding dimension
dropout = nn.Dropout(p=0.1)
layer_norm = nn.LayerNorm(encoder_output.size()[-1])
# We now will call the norm layer with our output
encoder_output = dropout(encoder_output)
encoder_output = layer_norm(encoder_output)
print(encoder_output)

tensor([[-0.1782,  1.6422, -1.0477, -0.4164],
        [-0.0688,  1.5921, -1.1488, -0.3746],
        [ 1.0643,  0.2894, -1.6430,  0.2894],
        [ 1.2054,  0.4652, -1.5191, -0.1515],
        [ 1.4756, -0.5458, -1.2038,  0.2741]],
       grad_fn=<NativeLayerNormBackward0>)


In [28]:
with torch.no_grad():
    printTensor(encoder_output)

          0         1         2         3
0 -0.178205  1.642216 -1.047657 -0.416354
1 -0.068807  1.592137 -1.148765 -0.374564
2  1.064282  0.289353 -1.642988  0.289353
3  1.205371  0.465171 -1.519063 -0.151480
4  1.475616 -0.545838 -1.203828  0.274050


In [16]:
#Keep this for the decoder implementation

def generate_attn_mask(input_seq, padding_idx):
    attn_mask = (input_seq != padding_idx).unsqueeze(1).unsqueeze(2)
    attn_mask = attn_mask.to(torch.float32).masked_fill(attn_mask == 0, float('-inf'))
    return attn_mask

If a tensor requires grads, use this method,
# Convert the tensor to a numpy array
tensor_numpy = tensor.cpu().numpy()

# Create a DataFrame using the numpy array
df = pd.DataFrame(tensor_numpy)

# Display the DataFrame
print(df)